In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [84]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,exquisite,impatiently,they,consequence,brighten,ear,there,authority,howl,private,...,prize,frame,choke,mutter,teacup,quadrille,i've,canterbury,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9865203761755486

Test set score: 0.9027255639097744


In [8]:
# Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with lasso.
# Question: Where do we use lasso here?
# LogisticRegression uses L2 by default, which is the same as Lasso

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3064) (3190,)
Training set score: 0.957680250783699

Test set score: 0.9163533834586466


In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.886833855799373

Test set score: 0.8735902255639098


In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
# Question: What are we doing here?
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)

# Question: What are we doing here?
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6976137211036539


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


In [15]:
pd.crosstab(y_Emma_test, lr_Emma_predicted).apply(lambda x: x / x.sum(), axis=1)

col_0,Austen,Carroll
row_0,,
Austen,0.937088,0.062912
Carroll,0.696940,0.303060


Well look at that! NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland. Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

---

**Carl note:** It looks lik ethe model is good at identifying Austen. Carroll, not so much.

## Challenge 0
Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 97%

In [16]:
# Get the baseline
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3064) (3190,)
Training set score: 0.957680250783699

Test set score: 0.9163533834586466


In [33]:
word_counts.columns

Index(['exquisite', 'impatiently', 'they', 'consequence', 'brighten', 'ear',
       'there', 'authority', 'howl', 'private',
       ...
       'choke', 'mutter', 'teacup', 'quadrille', 'i've', 'canterbury',
       'text_sentence', 'text_source', 'sentence_length',
       'sentence_punc_count'],
      dtype='object', length=3068)

In [17]:
# build new features
word_counts["sentence_length"] = [len(s) for s in word_counts["text_sentence"]]
word_counts["sentence_punc_count"] = word_counts["text_sentence"].apply(lambda x: sum([token.is_punct for token in x]))

In [34]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3066) (3190,)
Training set score: 0.9589341692789969

Test set score: 0.9177631578947368


In [19]:
# Note: This takes a long time
from sklearn import svm

clf = svm.SVC()
train = clf.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

(3190, 3066) (3190,)
Training set score: 0.6836990595611285

Test set score: 0.6917293233082706


In [38]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [87]:
df = sentences
df.columns = ["sentence", "author"]

In [88]:
df["document"] = [str(text) for text in df["sentence"]]

In [93]:
df.drop("sentence", axis=1, inplace=True)

In [114]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vec = TfidfVectorizer(max_features=1000, max_df=0.5)
document_vectorized = vec.fit_transform(df["document"])

//anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [115]:
y = df["author"]

In [116]:
X_train, X_test, y_train, y_test = train_test_split(document_vectorized, 
                                                    y,
                                                    test_size=0.4,
                                                    random_state=0)

In [117]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 1000) (3190,)
Training set score: 0.9275862068965517

Test set score: 0.8881578947368421


In [118]:
clf = svm.SVC()
train = clf.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

(3190, 1000) (3190,)
Training set score: 0.6824451410658308

Test set score: 0.6917293233082706


In [147]:
# Remove the word "lower" - we think it's messing with our model
df["document"] = df["document"].str.replace("lower", " ")

In [158]:
# PIPELINE ALL THE THINGS!

from sklearn.pipeline import Pipeline, make_pipeline

# pipe = Pipeline(steps=[('count', CountVectorizer()),
#                        ('tfidf', TfidfVectorizer(max_features=1000, max_df=0.5)),
#                        ('lr', LogisticRegression())])

# pipe.fit(df["document"], df["author"])
# pipe.score(df["document"], df["author"])


vec = CountVectorizer()
clf = LogisticRegression()
pipe = make_pipeline(vec, clf)
pipe.fit(df["document"], df["author"])

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [159]:
pipe.score(df["document"], df["author"])

0.9687852576156449

In [161]:
# JOIN FORCES - ENSEMBLE
# PIPELINE ALL THE THINGS!

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
ens = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svm', svm)])


vec = CountVectorizer()
pipe = make_pipeline(vec, ens)
pipe.fit(df["document"], df["author"])

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...l=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None))])

In [162]:
pipe.score(df["document"], df["author"])

//anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9710417450169236

In [163]:
X_train, X_test, y_train, y_test = train_test_split(df["document"], 
                                                    df["author"],
                                                    test_size=0.4,
                                                    random_state=0)

In [164]:
# JOIN FORCES - ENSEMBLE
# PIPELINE ALL THE THINGS!

from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
ens = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('svm', svm)])


vec = CountVectorizer()
pipe = make_pipeline(vec, ens)
pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...l=0.001, verbose=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None))])

In [165]:
pipe.score(X_train, y_train)

//anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9739811912225705

In [166]:
pipe.score(X_test, y_test)

//anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8641917293233082

## Challenge 1

Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [20]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [21]:
# Clean the Macbeth data.
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'VOLUME \w+', '', macbeth)
macbeth = re.sub(r'CHAPTER \w+', '', macbeth)
macbeth = text_cleaner(macbeth)
print(macbeth[:100])

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 


In [22]:
# Parse our cleaned data.
macbeth_doc = nlp(macbeth)

# Group into sentences.
macbeth_sents = [[sent, "Shakespear"] for sent in macbeth_doc.sents]

# Let's cut it down to the same length as Alice.
macbeth_sents = macbeth_sents[0:len(alice_sents)]

In [23]:
# Build a new Bag of Words data frame for Macbeth word counts.
# We'll use the same common words from Alice and Persuasion.
macbeth_sentences = pd.DataFrame(macbeth_sents)
macbeth_bow = bow_features(macbeth_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [24]:
macbeth_bow

,exquisite,impatiently,they,consequence,brighten,ear,there,authority,howl,private,...,prize,frame,choke,mutter,teacup,quadrille,i've,canterbury,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Actus, Primus, .)",Shakespear
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Scoena, Prima, .)",Shakespear
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Thunder, and, Lightning, .)",Shakespear
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Enter, three, Witches, .)",Shakespear
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(1, .)",Shakespear
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(When, shall, we, three, meet, againe, ?)",Shakespear
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, Thunder, ,, Lightning, ,, or, in, Raine, ?)",Shakespear
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(2, .)",Shakespear
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(When, the, Hurley, -, burley, 's, done, ,, Wh...",Shakespear
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(That, will, be, ere, the, set, of, Sunne, 1, .)",Shakespear


In [25]:
# Question: What is the difference of doing this like I did below v.s. what we did in the curriculum earlier (I pasted the curric code in the last cell)

In [26]:
text_to_compare = pd.concat([emma_bow, macbeth_bow])


Y = text_to_compare['text_source']
X = np.array(text_to_compare.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2002, 3064) (2002,)
Training set score: 0.9340659340659341

Test set score: 0.8375748502994012


In [27]:
# # Now we can model it!
# # Let's use logistic regression again.

# # Combine the Emma sentence data with the Alice data from the test set.
# # Question: What are we doing here?
# X_Emma_test = np.concatenate((
#     X_train[y_train[y_train=='Carroll'].index],
#     emma_bow.drop(['text_sentence','text_source'], 1)
# ), axis=0)

# # Question: What are we doing here?
# y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
#                          pd.Series(['Austen'] * emma_bow.shape[0])])

# # Model.
# print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
# lr_Emma_predicted = lr.predict(X_Emma_test)
# pd.crosstab(y_Emma_test, lr_Emma_predicted)